In [1]:
pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install colorama
!pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

class DialogueGenerator:
    def __init__(self, model_ref="gpt2"):
        try:
            self.tokenizer = GPT2Tokenizer.from_pretrained(model_ref)
            self.model = GPT2LMHeadModel.from_pretrained(model_ref)
            self.model.eval()
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(self.device)
        except Exception as e:
            print(f"Error loading model: {e}")


In [6]:
!pip install transformers datasets pandas --quiet

import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer
import random

# ✅ Step 1: Load CSV using Pandas
df = pd.read_csv("data_iconic.csv")

# ✅ Step 2: Clean the data
df = df[df["text"].notnull()]
df = df[df["text"].str.strip() != ""]

# ✅ Step 3: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["text"]])

# ✅ Step 4: Augmentation
def augment_dialogue(example):
    text = example["text"]
    tweaks = [
        lambda t: t.lower().capitalize(),
        lambda t: t.replace(".", "..."),
        lambda t: "👉 " + t,
        lambda t: t + " 🎬",
        lambda t: t.replace("I", "i") if t.startswith("I ") else t,
    ]
    tweak = random.choice(tweaks)
    return {"text": tweak(text)}

augmented_dataset = dataset.map(augment_dialogue)

# ✅ Step 5: Tokenization
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = augmented_dataset.map(tokenize_function, batched=True)

# ✅ Show one tokenized example
print(tokenized_dataset[0])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'text': '👉 I am Batman.', 'input_ids': [41840, 231, 314, 716, 9827, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer # Import GPT2Tokenizer

# Define tokenizer (copying from ShKHmLiC9Fj4)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


# ✅ Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Ensure tokenizer and model match

# ✅ Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling, since we're doing causal LM (GPT-2 style)
)

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./batman_gpt2_dialogue_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=200,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # Turn off wandb for now
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset, # Assuming tokenized_dataset is available from a previous cell
    data_collator=data_collator,
)

# 🚀 Start training
trainer.train()

# 💾 Save final model
model.save_pretrained("batman_gpt2_dialogue_model")
tokenizer.save_pretrained("batman_gpt2_dialogue_model")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.146900
20,3.393400
30,3.274500
40,2.588100
50,2.440700
60,2.443500
70,2.095600


('batman_gpt2_dialogue_model/tokenizer_config.json',
 'batman_gpt2_dialogue_model/special_tokens_map.json',
 'batman_gpt2_dialogue_model/vocab.json',
 'batman_gpt2_dialogue_model/merges.txt',
 'batman_gpt2_dialogue_model/added_tokens.json')

In [ ]:
import re
import random
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./batman_gpt2_dialogue_model")
tokenizer = GPT2Tokenizer.from_pretrained("./batman_gpt2_dialogue_model")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Cinematic emojis (controlled)
cinematic_emojis = ["🦇", "🎬", "🔥", "💥", "🃏", "🛡️", "😈"]

def clean_output(text):
    # Remove all non-alphabetic symbols except .?!,
    text = re.sub(r'[^a-zA-Z0-9\s\.\?\!]', '', text)
    # Remove duplicate spaces
    text = re.sub(r'\s{2,}', ' ', text)
    # Fix trailing cutoff
    if not text.endswith(('.', '?', '!')):
        text += '.'
    return text.strip()

# Input loop
while True:
    prompt = input("🎬 Enter your dialogue prompt (or 'exit' to quit): ")
    if prompt.lower() == "exit":
        break

    output = generator(
        prompt.strip().capitalize(),
        max_new_tokens=80,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )

    raw = output[0]["generated_text"]
    cleaned = clean_output(raw)


    print("\n🗣️ AI Dialogue:\n" + cleaned + " " )
    print("-" * 50)



Device set to use cpu



🗣️ AI Dialogue:
Im batmane hark. st. t t s s.t . 
--------------------------------------------------

🗣️ AI Dialogue:
Why so serious? I had to change the subject. I have the answers. I see. I am what I want to be. I just wanted to see . 
--------------------------------------------------

🗣️ AI Dialogue:
Your mission is to give us freedom. Im going to stop you you. I will not let you stand in my way. G.I. Joe. Dont call me Gwen bro. Youll never get me. Ill be back. 
--------------------------------------------------
